```
Last modified: 2021/09/26, @haewoon 
```


# Lab: Quantifying Gender Stereotypes in Word Embeddings

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/haewoon/lab-bias-in-word-embeddings/blob/master/Lab%20-%20Gender%20Stereotypes%20in%20Word%20Embeddings.ipynb)

## Step 0: Embeddind data download

The data (word embedding and occupation list) is prepared from https://github.com/tolga-b/debiaswe

In [ ]:
!gdown --id 1WfekqbKFmYdro8diMS-zpihwQ7CAq3ZE

In [ ]:
!unzip -o w2v_gnews_small.zip

## Step 1: Load data


### Word embedding
As the entire Google News embedding (https://code.google.com/archive/p/word2vec/) is too big to load, we use a small word embedding that contains only words that are required for this lab (what we downloaded in Step 0). 

In [ ]:
import numpy as np

from we import WordEmbedding

# load google news word2vec
E = WordEmbedding('w2v_gnews_small.txt')

### Occupations

Load a list of 320 occupations

In [ ]:
occupations = []
with open('occupations.txt') as fi:
    for line in fi:
        occupations.append(line.strip())
occupations[:5]

In [ ]:
len(occupations)

## Step 2: Define a gender axis vector (= *v(she)* - *v(he)*)

A gender axis can be defined as a difference between a vector of `she` and that of `he`. You can use multiple relevant nouns (e.g., man, mankind, son, male, etc.) to represent male and female by averging their vectors.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;`gender_axis`<br/>
`he` ------------------------> `she`

### *v(she)*: Vector of `she` in word embeddings

In [ ]:
len(E.v('she'))

In [ ]:
E.v('she')[:5] # presentaiton purpose. 5 out of 300 dimensions

### *v(he)*: Vector of `he` in word embeddings

In [ ]:
len(E.v('he'))

In [ ]:
E.v('he')[:5] # presentaiton purpose. 5 out of 300 dimensions

### *v_gender* = *v(she)* - *v(he)*: Gender axis vector

In [ ]:
v_gender = E.diff('she', 'he') # normalization is included
np.linalg.norm(v_gender)

In [ ]:
len(v_gender)

In [ ]:
v_gender[:5] # presentation purpose. 5 out of 300 dimensions

## Step 3: Analyzing gender bias in word embeddings with regard to occupations

We will compute the cosine similarity between a vector of each occupation and *v_gender*.

- similarity(v_gender, occupation) > 0 (well aligned with the gender axis): the occupation is closer to `she`.
- similarity(v_gender, occupation) < 0 (Opposite direction with the gender axis): the occupation is closer to `he`.

In [ ]:
similarities = []
for occupation in occupations:
    similarities.append((occupation, E.v(occupation).dot(v_gender)))

import operator
similarities = sorted(similarities, key=operator.itemgetter(1))

#### 20 occupation closer to `she` in word embeddings

In [ ]:
similarities[-20:]

#### 20 occupation closer to `he` in word embeddings

In [ ]:
similarities[:20]